In [2]:
from IPython.display import display 
import random
import json
import string
import os
from PIL import Image 

ImportError: dlopen(/Users/maghnus/Library/Python/3.8/lib/python/site-packages/PIL/_imaging.cpython-38-darwin.so, 0x0002): tried: '/Users/maghnus/Library/Python/3.8/lib/python/site-packages/PIL/_imaging.cpython-38-darwin.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/usr/local/lib/_imaging.cpython-38-darwin.so' (no such file), '/usr/lib/_imaging.cpython-38-darwin.so' (no such file)

In [ ]:

traits_dict = {}

# trait_types = os.listdir("trait-layers/")
trait_types = [
    '1|1',
    'canvas',
    'door',
    'background',
    'message',
    'type',
    'eyes',
    'beard',
    'right hand',
    'base',
    'headphones',
    'long hair',
    'hat under headphones',
    'short hair',
    'hat over headphones',
    'shirt',
    'smoke',
    'message'
]

specials = ["graveyard", "space", "tree"]

for trait_type in trait_types:
    if trait_type != ".DS_Store":
        traits_dict[trait_type] = [str.replace('.png', '').lower() for str in os.listdir("trait-layers/{}/".format(trait_type)) if str != ".DS_Store"]

In [ ]:
## Generate Traits

json_dict = {}

# get the actual mfers traits as JSON
with open("test.json") as json_raw:
    json_dict = json.load(json_raw)
    TOTAL_IMAGES = len(json_dict)

all_images = [] 

# A recursive function to generate unique image combinations
def create_new_image(tokenid):
    
    new_image = {}
    
#     Now a list of the form [{'trait_type': 'background', 'value': 'red'}] etc
    if "{}".format(tokenid) in json_dict:
        mfer_attrs = json_dict["{}".format(tokenid)]["attributes"]
    else:
        mfer_attrs = [{'trait_type': '1|1', 'value': 'None'}]
        
    print("mfer attrs")
    print(mfer_attrs)
    
    for dic in mfer_attrs:
        
        temp = dic["trait_type"]
        temp = temp.replace("/", "|")
        dic["trait_type"] = temp.lower()
        
        if dic["trait_type"] in traits_dict:
            
            temp = dic["value"]
            dic["value"] = temp.lower()
            
            # replace word mfer with dadmfer
            if dic["trait_type"] == "type":
                temp = dic["value"]
                dic["value"] = temp.replace(" mfer", " dadmfer")
                print(dic["value"])
                
            if dic["trait_type"] == "1|1":
                new_image[dic["trait_type"]] = "None" # "''.join(random.choice(string.ascii_lowercase) for i in range(20))
                
            # replace / with |
            if "/" in dic["value"]:
                temp = dic["value"]
                dic["value"] = temp.replace("/", "|")
            
            
            if dic["value"] in traits_dict[dic["trait_type"]]:
                new_image[dic["trait_type"]] = dic["value"]
            else:
                print(mfer_attrs)
                print("dont have value {} with trait {}".format(dic["value"], dic["trait_type"]))
        else:
            print(mfer_attrs)
            print("dont have type {}".format(dic["trait_type"]))
    
    new_image["base"] = "skeleton"
    avail_canvases = traits_dict["canvas"].copy()
    
    print("-----")
    print(tokenid)
    print(mfer_attrs)
    print(new_image)
    print("-----")
    
    if "1|1" in new_image:
        new_image["background"] = "orange"
        new_image["canvas"] = "orange"
        new_image["door"] = "orange"
        new_image["message"] = "klingon"
        
    else:
    
        # canvas cannot match background
        if new_image["background"] in avail_canvases:
            avail_canvases = [name for name in avail_canvases if name not in specials]

            if new_image["background"] not in specials:
                avail_canvases.remove(new_image["background"])

            new_image["canvas"] = random.choice(avail_canvases)

        avail_doors = traits_dict["door"].copy()
        if new_image["background"] in avail_doors:
            avail_doors = [name for name in avail_doors if name not in specials]

            if new_image["background"] not in specials:
                avail_doors.remove(new_image["background"])

            print(new_image["canvas"])
            avail_doors.remove(new_image["canvas"])
            new_image["door"] = random.choice(avail_doors)

        # MESSAGE
        new_image["message"] = "dads gotta know!"

        if new_image["type"] == "zombie dadmfer":
            new_image["canvas"] = "graveyard"
            new_image["door"] = "graveyard"
            new_image["message"] = "back from the ded"

        if new_image["type"] == "alien dadmfer":
            new_image["canvas"] = "space"
            new_image["door"] = "space"
            new_image["message"] = "klingon"

        if new_image["type"] == "ape dadmfer":
            new_image["canvas"] = "tree"
            new_image["door"] = "tree"
            new_image["message"] = "sus talk"

        # RIGHT HAND
        new_image["right hand"] = random.choice(traits_dict["right hand"])

    return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(0, TOTAL_IMAGES): 
    
    new_trait_image = create_new_image(i)
    
    all_images.append(new_trait_image)
    


In [ ]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

In [ ]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [ ]:
# Get Trait Counts

counts = {}

for k in traits_dict:
    counts[k] = {}
    for item in traits_dict[k]:
        counts[k][item] = 0
        
for image in all_images:
    for k in traits_dict:
        if k in image:
#             TODO REMEMBER THAT YOU TOOK OUT SPECIAL BACKGROUNDS ETC
            if k in counts and image[k] in counts[k]:
                counts[k][image[k]] += 1

In [ ]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [ ]:
  
    
#### Generate Images    
for item in all_images[6580:]:
    
    print("item")
    print(item)
    
    image_list = []
    
    max = 0
    for k in trait_types:
        if k in item:
            image_list += [Image.open(f'./trait-layers/{k}/{item[k].lower()}.png').convert('RGBA')]
            max += 1

    #Create each composite
    
    comp = Image.alpha_composite(image_list[0], image_list[1])
    for i in range(2, max):
        comp = Image.alpha_composite(comp, image_list[i])

    #Convert to RGB
    rgb_im = comp.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)
    
    
    

In [ ]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
#     token["attributes"].append(getAttribute("Background", i["Background"]))
#     token["attributes"].append(getAttribute("Circle", i["Circle"]))
#     token["attributes"].append(getAttribute("Square", i["Square"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()


    # HEAD CHOICES
#     head_type = random.choice([1, 2, 3, 4])
#     if head_type == 1:
#         # keep short hair
#         new_image.pop("Hat over headphones")
#         new_image.pop("Hat under headphones")
#         new_image.pop("Long hair")
#     if head_type == 2:
#         # keep long hair
#         new_image.pop("Hat over headphones")
#         new_image.pop("Hat under headphones")
#         new_image.pop("Short hair")
#     if head_type == 3:
#         # keep over headphones
#         new_image.pop("Short hair")
#         new_image.pop("Hat under headphones")
#         new_image.pop("Long hair")
#     if head_type == 4:
#         # keep under headphones
#         new_image.pop("Short hair")
#         new_image.pop("Hat over headphones")
#         new_image.pop("Long hair")
        
    
    # APE / ZOMBIE / ALIEN COHESIVENESS
    
#     if new_image["Type"] == "Alien dadmfer":
#         new_image["Background"] = "Alien"
#     if new_image["Type"] == "Zombie dadmfer":
#         new_image["Background"] = "Zombie"
#     if new_image["Type"] == "Ape dadmfer":
#         new_image["Background"] = "Tree"
        
    # OTHER RULES
    
#     if "Hat over headphones" in new_image:
#         if new_image["Hat over headphones"] == "Pilot helmet":
#             new_image.pop("Headphones")

    # For each trait category, select a random trait based on the weightings 